# Sequence Models & How to use them

**학습목표**
1. RNN, LSTM, GRU의 구조를 이해한다.
2. 위 모델을 사용하기 위한 데이터 구조를 이해한다.
3. 위 모든 내용을 코딩할 수 있다.

-----------------
* Thanks to : 한기영 대표님 @ Data Insight

## 1.환경 및 데이터 준비

### Import Packages

In [2]:
#라이브러리들을 불러오자.
import tensorflow as tf
from tensorflow import keras

import numpy as np
import pandas as pd


### Data Loading

In [3]:
url = 'https://raw.githubusercontent.com/RayleighKim/Example_datasets/master/Stock_Edwards_Lifesciences_corporation.csv'

# 판다스로 데이터를 불러오시오.
data = pd.read_csv(url)

In [4]:
data.head()

,Date,Open,High,Low,Close,Adj_Close,Volume
0,2000/3/27,3.812500,4.156250,3.812500,4.125000,4.125000,3675600
1,2000/3/28,4.125000,4.125000,4.000000,4.015625,4.015625,1077600
2,2000/3/29,4.000000,4.031250,3.953125,4.000000,4.000000,437200
3,2000/3/30,4.000000,4.000000,3.843750,3.843750,3.843750,1883600
4,2000/3/31,3.734375,3.734375,3.390625,3.390625,3.390625,7931600


##3.데이터 준비

### Date 컬럼을 제거하시오.

In [5]:
data = data.drop('Date', axis = 1)

In [6]:
data.head()

,Open,High,Low,Close,Adj_Close,Volume
0,3.812500,4.156250,3.812500,4.125000,4.125000,3675600
1,4.125000,4.125000,4.000000,4.015625,4.015625,1077600
2,4.000000,4.031250,3.953125,4.000000,4.000000,437200
3,4.000000,4.000000,3.843750,3.843750,3.843750,1883600
4,3.734375,3.734375,3.390625,3.390625,3.390625,7931600


In [7]:
data.corr()

,Open,High,Low,Close,Adj_Close,Volume
Open,1.000000,0.999907,0.999899,0.999806,0.999806,0.048770
High,0.999907,1.000000,0.999874,0.999909,0.999909,0.051444
Low,0.999899,0.999874,1.000000,0.999912,0.999912,0.045101
Close,0.999806,0.999909,0.999912,1.000000,1.000000,0.047917
Adj_Close,0.999806,0.999909,0.999912,1.000000,1.000000,0.047917
Volume,0.048770,0.051444,0.045101,0.047917,0.047917,1.000000


In [8]:
# data = data.drop('Volume', axis = 1)

## 4.Sequence 데이터 구조로 만들기

**조건**
* 내일의 Close를 예측할 것이다. 이를 Y로 둘 것.
* timestep은 4주를 본다. (주식은 5일이 1주일)
* 맞추어 전처리

**데이터 분할 규칙**
* 가장 최근 1주일을 테스트 데이터로 둔다.
* 테스트 데이터를 제외하고, 가장 최근 2주를 벨리데이션 데이터로 둔다.

In [9]:
data.head()

,Open,High,Low,Close,Adj_Close,Volume
0,3.812500,4.156250,3.812500,4.125000,4.125000,3675600
1,4.125000,4.125000,4.000000,4.015625,4.015625,1077600
2,4.000000,4.031250,3.953125,4.000000,4.000000,437200
3,4.000000,4.000000,3.843750,3.843750,3.843750,1883600
4,3.734375,3.734375,3.390625,3.390625,3.390625,7931600


In [10]:
# 판다스 데이터 프레임을 넘파이로 옮김.
data = data.values

In [11]:
data.shape

(4392, 6)

In [12]:
timestep= 20 

x = [ ]  # x를 담을 빈 공간
y = [ ]  # y를 담을 빈 공간

x = np.array([data[i:i+timestep] for i in range(len(data) - timestep)])
y = np.array([data[i+timestep, -3] for i in range(len(data) - timestep)])

x_train, x_val, x_test = x[:-15], x[-15:-5], x[-5:]
y_train, y_val, y_test = y[:-15], y[-15:-5], y[-5:]

print(x.shape, y.shape)
print('-------------------------------')
print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)
print(x_test.shape, y_test.shape)
print('-------------------------------')
print('[ # , timestep, feature수 ] <-- 데이터의 구조 : ')

(4372, 20, 6) (4372,)
-------------------------------
(4357, 20, 6) (4357,)
(10, 20, 6) (10,)
(5, 20, 6) (5,)
-------------------------------
[ # , timestep, feature수 ] <-- 데이터의 구조 : 


데이터의 구조 : [n, timestep, feature수]

# Q.1. 단순 RNN복습

1. 적절한 인풋 레이어를 구성할 것
2. 첫번째 히든레이어 : SimpleRNN
    * 히든스테이트 노드 16개, return_sequences=True
3. 두번째 히든레이어 : SimpleRNN
    * 히든스테이트 노드 32개, return_sequences=True
4. 플래튼
5. 회귀를 위한 적절한 아웃풋 레이어

In [13]:
x_train.shape, y_train.shape

((4357, 20, 6), (4357,))

In [14]:
from keras.backend import clear_session
from keras.layers import Input, Dense, Flatten, SimpleRNN
from keras.models import Model

In [18]:
####################
## Your Code Here ##
####################
# 1. 이전 세션 클리어
clear_session()
# 2. 모델 블록 쌓기
il = Input(shape = (20, 6))

hl = SimpleRNN(16, return_sequences= True)(il)
hl = SimpleRNN(32, return_sequences= True)(hl)
hl = Flatten()(hl)

ol = Dense(1)(hl)
# 3. 모델 선언
model = Model(il, ol)
# 4. 모델 컴파일
model.compile(loss='mae', optimizer = 'adam')
# 모델 요약
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20, 6)]           0         
                                                                 
 simple_rnn (SimpleRNN)      (None, 20, 16)            368       
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 20, 32)            1568      
                                                                 
 flatten (Flatten)           (None, 640)               0         
                                                                 
 dense (Dense)               (None, 1)                 641       
                                                                 
Total params: 2,577
Trainable params: 2,577
Non-trainable params: 0
_________________________________________________________________


In [16]:
from keras.callbacks import EarlyStopping

es = EarlyStopping(monitor = 'val_loss',
                   min_delta = 0,
                   patience = 5,
                   restore_best_weights = True)

In [19]:
## 학습도 시킬 것
model.fit(x_train, y_train, epochs = 1000, callbacks = [es], validation_data=(x_val, y_val), verbose = 1)


Epoch 1/1000
137/137 [==============================] - 3s 12ms/step - loss: 21.9253 - val_loss: 97.6927
Epoch 2/1000
137/137 [==============================] - 1s 9ms/step - loss: 21.4091 - val_loss: 96.7622
Epoch 3/1000
137/137 [==============================] - 1s 9ms/step - loss: 21.4318 - val_loss: 98.8533
Epoch 4/1000
137/137 [==============================] - 1s 9ms/step - loss: 21.4080 - val_loss: 98.4993
Epoch 5/1000
137/137 [==============================] - 2s 14ms/step - loss: 21.4384 - val_loss: 97.6449
Epoch 6/1000
137/137 [==============================] - 1s 9ms/step - loss: 21.4486 - val_loss: 97.6906
Epoch 7/1000
137/137 [==============================] - 1s 9ms/step - loss: 21.4147 - val_loss: 99.5141


In [20]:
# Q1. 테스트 셋에서의 RMSE를 출력하여라.
from sklearn.metrics import mean_squared_error as MSE

y_pred = model.predict(x_test)

RMSE = MSE(y_test, y_pred) ** 0.5
print(RMSE)

1/1 [==============================] - 0s 261ms/step
96.34585034150095


# Q.2. LSTM, GRU 복습

1. 적절한 인풋 레이어를 구성할 것
2. 첫번째 히든레이어 : LSTM
    * 히든스테이트 노드 16개, return_sequences=True
3. 두번째 히든레이어 : GRU
    * 히든스테이트 노드 32개, return_sequences=True
4. 플래튼
5. Fully Connected Layer, 노드 128개, swish
5. 회귀를 위한 적절한 아웃풋 레이어

In [21]:
from keras.backend import clear_session
from keras.layers import Input, Dense, Flatten, LSTM, GRU
from keras.models import Model

In [22]:
x_train.shape, y_train.shape

((4357, 20, 6), (4357,))

In [23]:
####################
## Your Code Here ##
####################
# 1. 이전 세션 클리어
clear_session()
# 2. 모델 엮기
il = Input(shape = (20, 6))

hl = LSTM(16, return_sequences = True)(il)
hl = GRU(32, return_sequences = True)(hl)
hl = Flatten()(hl)
hl = Dense(128, activation = 'swish')(hl)

ol = Dense(1)(hl)
# 3. 모델 생성
model = Model(il, ol)
# 4. 컴파일
model.compile(loss = 'mae', optimizer = 'adam')
# 모델 요약
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20, 6)]           0         
                                                                 
 lstm (LSTM)                 (None, 20, 16)            1472      
                                                                 
 gru (GRU)                   (None, 20, 32)            4800      
                                                                 
 flatten (Flatten)           (None, 640)               0         
                                                                 
 dense (Dense)               (None, 128)               82048     
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 88,449
Trainable params: 88,449
Non-trainable p

In [24]:
from keras.callbacks import EarlyStopping

es = EarlyStopping(monitor = 'val_loss',
                   min_delta = 0,
                   patience = 7,
                   restore_best_weights = True)

In [25]:
## 학습도 시킬 것
model.fit(x_train, y_train, epochs = 1000, callbacks = [es], validation_data=(x_val, y_val), verbose = 1)


Epoch 1/1000
137/137 [==============================] - 8s 27ms/step - loss: 22.1497 - val_loss: 97.6298
Epoch 2/1000
137/137 [==============================] - 4s 32ms/step - loss: 21.4154 - val_loss: 97.6255
Epoch 3/1000
137/137 [==============================] - 3s 21ms/step - loss: 21.5270 - val_loss: 97.1980
Epoch 4/1000
137/137 [==============================] - 3s 21ms/step - loss: 21.4262 - val_loss: 98.9868
Epoch 5/1000
137/137 [==============================] - 3s 21ms/step - loss: 21.4470 - val_loss: 99.0392
Epoch 6/1000
137/137 [==============================] - 4s 26ms/step - loss: 21.4183 - val_loss: 98.5868
Epoch 7/1000
137/137 [==============================] - 3s 21ms/step - loss: 21.4368 - val_loss: 100.0556
Epoch 8/1000
137/137 [==============================] - 3s 21ms/step - loss: 21.4403 - val_loss: 97.8466
Epoch 9/1000
137/137 [==============================] - 3s 21ms/step - loss: 21.4574 - val_loss: 97.4531
Epoch 10/1000
137/137 [==============================]

In [26]:
y_pred = model.predict(x_test)

1/1 [==============================] - 1s 732ms/step


In [27]:
# Q1. 테스트 셋에서의 RMSE를 출력하여라.
from sklearn.metrics import mean_squared_error

RMSE = mean_squared_error(y_test, y_pred) ** 0.5
print(RMSE)

96.78161771909474


In [28]:
x_train.shape, y_train.shape

((4357, 20, 6), (4357,))

In [38]:
from keras.layers import Dense, Flatten, SimpleRNN
from keras.layers import Input, LSTM, GRU
from keras.layers import Bidirectional, Conv1D, MaxPool1D

from keras.models import Sequential, Model

from keras.backend import clear_session

In [41]:
clear_session()

il = Input(shape = (20, 6))

hl = Conv1D(filters = 16, # 16종류의 특징을 제작해줘
            kernel_size = 5, # 한번에 5시점씩 고려해줘야해
            activation = 'relu',
            padding = 'same')(il)
hl = Conv1D(filters = 16, # 16종류의 특징을 제작해줘
            kernel_size = 5, # 한번에 5시점씩 고려해줘야해
            activation = 'relu',
            padding = 'same')(il)
hl = MaxPool1D(2)(hl) # 배치놈과 드랍아웃과 시퀀스 데이터 <-- 셍각해보기
hl = Bidirectional(layer = LSTM(32, return_sequences= True))(hl)


forward_gru = GRU(32, return_sequences=True)
backward_LSTM = LSTM(24, return_sequences=True, go_backwards=True)
hl = Bidirectional(layer = forward_gru, backward_layer = backward_LSTM)(hl)

hl = Flatten()(hl)


ol = Dense(1,activation = 'relu')(hl) # 무슨 효과가 있을까요?

model = Model(il, ol)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20, 6)]           0         
                                                                 
 conv1d_1 (Conv1D)           (None, 20, 16)            496       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 10, 16)           0         
 )                                                               
                                                                 
 bidirectional (Bidirectiona  (None, 10, 64)           12544     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 10, 56)           17952     
 nal)                                                            
                                                             